In [1]:
import pickle

with open('kana_chars.pickle', mode='rb') as f:
    chars_list = pickle.load(f)
print(chars_list)

def is_invalid(message):
    is_invalid = False
    for char in message:
        if char not in chars_list:
            is_invalid = True
    return is_invalid

['\t', '\n', '、', '。', '々', 'ぁ', 'あ', 'ぃ', 'い', 'ぅ', 'う', 'ぇ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た', 'だ', 'ち', 'ぢ', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ', 'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'ゎ', 'わ', 'ゐ', 'ゑ', 'を', 'ん', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'エ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'ゲ', 'コ', 'ゴ', 'サ', 'ザ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ゼ', 'ソ', 'ゾ', 'タ', 'ダ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'テ', 'デ', 'ト', 'ド', 'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', 'ハ', 'バ', 'パ', 'ヒ', 'ビ', 'ピ', 'フ', 'ブ', 'プ', 'ヘ', 'ベ', 'ペ', 'ホ', 'ボ', 'ポ', 'マ', 'ミ', 'ム', 'メ', 'モ', 'ャ', 'ヤ', 'ュ', 'ユ', 'ョ', 'ヨ', 'ラ', 'リ', 'ル', 'レ', 'ロ', 'ヮ', 'ワ', 'ヰ', 'ヱ', 'ヲ', 'ン', 'ヴ', '・', 'ー', '？']


In [2]:
import numpy as np

char_indices = {}
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char={}
for i, char in enumerate(chars_list):
    indices_char[i] = char
    
n_char = len(chars_list)
max_length_x = 128

def sentence_to_vector(sentence):
    vector = np.zeros((1 , max_length_x, n_char), dtype=np.bool)
    for j, char in enumerate(sentence):
        vector[0][j][char_indices[char]] = 1
    return vector

In [3]:
from keras.models import load_model

encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

def respond(message, beta=5):
    vec = sentence_to_vector(message)
    state_value = encoder_model.predict(vec)
    y_decoder = np.zeros((1, 1, n_char))
    y_decoder[0][0][char_indices["\t"]] = 1
    
    respond_sentence = ""
    while True:
        y, h = decoder_model.predict([y_decoder, state_value])
        p_power = y[0][0] ** beta
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))
        next_char = indices_char[next_index]
        
        if (next_char == "\n" or len(respond_sentence) >= max_length_x):
            break
            
        respond_sentence += next_char
        y_decoder = np.zeros((1, 1, n_char))
        y_decoder[0][0][next_index] = 1
        
        state_value = h
        
    return respond_sentence

Using TensorFlow backend.
C:\Users\NDA04\Anaconda3\envs\nlp_bot\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
bot_name = "賢治bot"
your_name = input("おなまえをおあしえてください。:")
print()

print(bot_name + ": " + "こんにちは、"+ your_name + "さん。")
message = ""
while message != "さようなら。":
    
    while True:
        message = input(your_name + ": ")
        if not is_invalid(message):
            break
        else:
            print(bot_name + ": ひらがなか、カタカナをつかってください。")
            
    response = respond(message)
    print(bot_name + ": " + response)

おなまえをおあしえてください。:nobu

賢治bot: こんにちは、nobuさん。
nobu: こんにちは、けんじさん
賢治bot: なんだかあったのです。
nobu: うまくいかないのです
賢治bot: おまえはもうだんだんはいっていました。
nobu: わからなくなってきました
賢治bot: このちょうどはいいろのかわのみずのなかにはいっていました。
nobu: かわあそびもたのしそうですね
賢治bot: きょうがいいました。
nobu: そうなんですね
賢治bot: そうだ。
nobu: いいことありますか
賢治bot: さんにんのあおいろいろのはらをみました。
nobu: さようなら
賢治bot: またさぶろうはすっかりきれいないから、いちばんはじめているのです。
nobu: さようなら。
賢治bot: またさぶろうはすっかりあげていきました。


In [5]:
bot_name1 = "賢治bot1"
bot_name2 = "賢治bot2"
your_name = input("おなまえをおあしえてください。:")
print()

print(bot_name1 + ": " + "こんにちは、"+ your_name + "さん。")
print(bot_name2 + ": " + "こんにちは、"+ your_name + "さん。")

message = ""
while message != "さようなら。":
    
    while True:
        message = input(your_name + ": ")
        if not is_invalid(message):
            break
        else:
            print(bot_name1 + ": ひらがなか、カタカナをつかってください。")
            
    response = respond(message)
    print(bot_name1 + ": " + response)
    print(bot_name2 + ": " + response)    

おなまえをおあしえてください。:nobu

賢治bot1: こんにちは、nobuさん。
賢治bot2: こんにちは、nobuさん。
nobu: 調子はどうですか？
賢治bot1: ひらがなか、カタカナをつかってください。
nobu: ちょうしはどうですか
賢治bot1: するとあたまをおとうさんのとうりのようになって、とうとうからからからだをだしているのです。
賢治bot2: するとあたまをおとうさんのとうりのようになって、とうとうからからからだをだしているのです。
nobu: さようなら。
賢治bot1: またさぶろうはすっかりきたのです。
賢治bot2: またさぶろうはすっかりきたのです。
